In [1]:
#Elsevier Libraries
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

#Libraries for URL access
import json
import csv
import pprint
import requests
import xmltodict
import urllib3
import numpy as np
import pandas as pd
#Import warnings library
import warnings
warnings.filterwarnings("ignore")

In [2]:
#xml.etree.ElementTree — The ElementTree XML API
import xml.etree.ElementTree as ET

In [3]:
info_list = [] #Creating an empty list to store the  info
#Rerunning this will delete the data that will be stored further
#Run only once

In [4]:
#Create a num array of 0 to 5900 spaces 100 apart per page. Each page of Search returns 100 entries until 6000 is reached
num = np.linspace(0, 6000, 100, dtype = int)
print(num)
#Maximum of 6000 searches are returned on calling the ElsSearch API

[   0   60  121  181  242  303  363  424  484  545  606  666  727  787
  848  909  969 1030 1090 1151 1212 1272 1333 1393 1454 1515 1575 1636
 1696 1757 1818 1878 1939 2000 2060 2121 2181 2242 2303 2363 2424 2484
 2545 2606 2666 2727 2787 2848 2909 2969 3030 3090 3151 3212 3272 3333
 3393 3454 3515 3575 3636 3696 3757 3818 3878 3939 4000 4060 4121 4181
 4242 4303 4363 4424 4484 4545 4606 4666 4727 4787 4848 4909 4969 5030
 5090 5151 5212 5272 5333 5393 5454 5515 5575 5636 5696 5757 5818 5878
 5939 6000]


In [5]:
#Append "apikey" and "insttoken" as suggest in the ElsSearch document into a config file
config = {
    "apikey": "2dc442325fc67f2f275ec3157ef8df65",
 "insttoken": "6beb1f6c29d85f50029bf11c8de94d1b"
    }

client = ElsClient(config['apikey'])
client.inst_token = config['insttoken']

In [6]:
def XML_DOI(link):
    
    #defining a header's dictionary to pass through requests
    headers_dict = {"X-ELS-APIKey": "2dc442325fc67f2f275ec3157ef8df65", "X-ELS-Insttoken": "6beb1f6c29d85f50029bf11c8de94d1b", "Accept": "application/xml"}
    
    #x takes response of the HTTP request, passes link
    x = requests.get(link, headers=headers_dict)
    
    #Save it as XML file
    with open("doi.xml", 'wb') as f:
        f.write(x.content)

In [10]:
link_list = []

#Loop to iterate over all papers
#UPI query can take many arguments.
#start, count, query are a few as name suggests
for i in range(0, 60):
    
    start = "https://api.elsevier.com/content/search/sciencedirect?" 
    count = "start=" + str(num[i]) + "&count=100"
    query = "&query=polyethylene+recycle+methods"
    endapi = "&apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b"
    
    link = start + count + query  + endapi 
    #print(link)
    link_list.append(link)
#subscription = "&subscribed=true"

In [11]:
link_list[50]

'https://api.elsevier.com/content/search/sciencedirect?start=3030&count=100&query=polyethylene+recycle+methods&apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b'

In [13]:
# Call the XML_DOI function
for j in range(0, 60):
    xmlfile = XML_DOI(link_list[j])
    #Read the data
    #<?xml version="1.0" encoding="UTF-8"?>
    tree = ET.parse(r'/home/user3/Documents/avan_phd/Objective_3/Objective_3_QnA/data_making_qna/doi.xml')
    root = tree.getroot()
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
    
        info_dict = {} #This will create a dictionary where we will store information about the searches
    
        '''
        CHECK CODE
        url = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        title = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        pub_name = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi').text
        #description = entry.find('{http://purl.org/dc/elements/1.1/}description').text
        print(url, title, pub_name, doi)
        print('\n')
        
        '''
    
        info_dict['URL'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}url').text
        info_dict['Title'] = entry.find('{http://purl.org/dc/elements/1.1/}title').text
        info_dict['Pub_Name'] = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}publicationName').text
        doi = entry.find('{http://prismstandard.org/namespaces/basic/2.0/}doi')
        if doi is None:
            info_dict['DOI'] = None
        else:
            info_dict['DOI'] = doi.text
    
        info_list.append(info_dict)
    
    #print(info_list)

In [14]:
type(info_list), len(info_list)

(list, 6300)

In [15]:
DOI =  []
Title = []
Pub_name = []

for i in range(6000):
    DOI.append(info_list[i]['DOI'])
    Title.append(info_list[i]['Title'])
    Pub_name.append(info_list[i]['Pub_Name'])

In [16]:
df =  pd.DataFrame()
df['Title'] =  Title
df['Pub_name'] = Pub_name
df['DOI'] = DOI

df.head()

Title  \
0  Environmental assessment of the recycling proc...   
1  Sustainable utilization of hybrid recycled pow...   
2  Simulations mapping the influence of oxygen, e...   
3  Migration of volatile substances from recycled...   
4  Effect of compatibilization agents and rice hu...   

                                 Pub_name                            DOI  
0  Sustainable Materials and Technologies   10.1016/j.susmat.2022.e00562  
1         Journal of Building Engineering     10.1016/j.jobe.2022.105541  
2                                 Polymer  10.1016/j.polymer.2022.125596  
3           Food Packaging and Shelf Life     10.1016/j.fpsl.2022.101020  
4                          Energy Reports     10.1016/j.egyr.2022.07.101

In [17]:
#Create string1 and string2 to join doi with institoken to make a single URL
string1 = "https://api.elsevier.com/content/article/doi/" 
string2 = "?apiKey=2dc442325fc67f2f275ec3157ef8df65&insttoken=6beb1f6c29d85f50029bf11c8de94d1b"

#Access every DOI in the previous file and append the new URL to another column
df['Link'] = df['DOI'].apply(lambda x: string1 + str(x) + string2)

df.head()

Title  \
0  Environmental assessment of the recycling proc...   
1  Sustainable utilization of hybrid recycled pow...   
2  Simulations mapping the influence of oxygen, e...   
3  Migration of volatile substances from recycled...   
4  Effect of compatibilization agents and rice hu...   

                                 Pub_name                            DOI  \
0  Sustainable Materials and Technologies   10.1016/j.susmat.2022.e00562   
1         Journal of Building Engineering     10.1016/j.jobe.2022.105541   
2                                 Polymer  10.1016/j.polymer.2022.125596   
3           Food Packaging and Shelf Life     10.1016/j.fpsl.2022.101020   
4                          Energy Reports     10.1016/j.egyr.2022.07.101   

                                                Link  
0  https://api.elsevier.com/content/article/doi/1...  
1  https://api.elsevier.com/content/article/doi/1...  
2  https://api.elsevier.com/content/article/doi/1...  
3  https://api.elsevier.com/content/article/doi/1...  
4  https://api.elsevier.com/content/article/doi/1...

In [18]:
df.head()

Title  \
0  Environmental assessment of the recycling proc...   
1  Sustainable utilization of hybrid recycled pow...   
2  Simulations mapping the influence of oxygen, e...   
3  Migration of volatile substances from recycled...   
4  Effect of compatibilization agents and rice hu...   

                                 Pub_name                            DOI  \
0  Sustainable Materials and Technologies   10.1016/j.susmat.2022.e00562   
1         Journal of Building Engineering     10.1016/j.jobe.2022.105541   
2                                 Polymer  10.1016/j.polymer.2022.125596   
3           Food Packaging and Shelf Life     10.1016/j.fpsl.2022.101020   
4                          Energy Reports     10.1016/j.egyr.2022.07.101   

                                                Link  
0  https://api.elsevier.com/content/article/doi/1...  
1  https://api.elsevier.com/content/article/doi/1...  
2  https://api.elsevier.com/content/article/doi/1...  
3  https://api.elsevier.com/content/article/doi/1...  
4  https://api.elsevier.com/content/article/doi/1...

In [19]:
%%time
list_abstract = []
list_doi= []
list_title = []
list_date = []
list_journal = []

## ScienceDirect (full-text) document example using DOI
#for i in range(len(data)):
from pprint import pprint
doi= df['DOI']
for i in range(df.shape[0]):
    pprint(doi[i])
    doi_doc = FullDoc(doi = doi[i])
    if doi_doc.read(client):

        #pprint(dir(doi_doc))
        #pprint(doi_doc._data)
        abstract = doi_doc._data['coredata']['dc:description']
        title = doi_doc.title
        date = doi_doc._data['coredata']['prism:coverDisplayDate']
        journal = doi_doc._data['coredata']['prism:publicationName']
        #pprint(abstract)
        print(i, 'done')
        #dict_abstract['DOI'] = doi[i]
        #dict_abstract['Abstract'] = abstract
        list_abstract.append(abstract)
        list_doi.append(doi[i])
        list_title.append(title)
        list_date.append(date)
        list_journal.append(journal)
        #doi_doc.write()
    
    else:
        pprint('Operation failed')
        #dict_abstract['DOI'] = doi[i]
        #dict_abstract['Abstract'] = 'NAN'
        list_abstract.append('0')
        list_doi.append('1')
        list_title.append('2')
        list_date.append('3')
        list_journal.append('4')

'10.1016/j.susmat.2022.e00562'
0 done
'10.1016/j.jobe.2022.105541'
1 done
'10.1016/j.polymer.2022.125596'
2 done
'10.1016/j.fpsl.2022.101020'
3 done
'10.1016/j.egyr.2022.07.101'
4 done
'10.1016/j.resconrec.2022.106444'
5 done
'10.1016/j.jics.2022.100843'
6 done
'10.1016/j.clwas.2022.100044'
7 done
'10.1016/j.jmrt.2022.10.126'
8 done
'10.1016/j.conbuildmat.2021.126118'
9 done
'10.1016/j.fpsl.2022.100890'
10 done
'10.1016/j.polymdegradstab.2023.110263'
11 done
'10.1016/j.conbuildmat.2022.129164'
12 done
'10.1016/j.crgsc.2022.100280'
13 done
'10.1016/j.conbuildmat.2022.127069'
14 done
'10.1016/j.jclepro.2022.134442'
15 done
'10.1016/j.jclepro.2022.132625'
16 done
'10.1016/j.jes.2022.09.039'
17 done
'10.1016/j.jclepro.2022.134950'
18 done
'10.1016/j.matpr.2022.03.293'
19 done
'10.1016/j.dib.2021.107452'
20 done
'10.1016/j.resconrec.2022.106639'
21 done
'10.1016/j.matpr.2022.03.385'
22 done
'10.1016/j.chemosphere.2022.136050'
23 done
'10.1016/j.supflu.2021.105503'
24 done
'10.1016/j.polymde

201 done
'10.1016/j.cej.2021.131344'
202 done
'10.1016/j.jclepro.2015.08.067'
203 done
'10.1016/j.jece.2022.108332'
204 done
'10.1016/j.jhazmat.2018.05.046'
205 done
'10.1016/j.conbuildmat.2022.130279'
206 done
'10.1016/j.matpr.2018.07.046'
207 done
'10.1016/j.rcradv.2022.200127'
208 done
'10.1016/j.jenvman.2020.110149'
209 done
'10.1016/j.polymertesting.2017.11.015'
210 done
'10.1016/j.matpr.2017.12.161'
211 done
'10.1016/j.jclepro.2022.134083'
212 done
'10.1016/j.cej.2017.08.058'
213 done
'10.1016/j.jenvman.2015.04.036'
214 done
'10.1016/j.jobe.2022.105445'
215 done
'10.1016/j.proeng.2016.06.016'
216 done
'10.1016/j.compstruct.2017.05.076'
217 done
'10.1016/j.indcrop.2018.09.012'
218 done
'10.1016/j.mineng.2021.107241'
219 done
'10.1016/j.jclepro.2014.02.009'
220 done
'10.1016/j.jmrt.2018.11.015'
221 done
'10.1016/j.wasman.2018.02.045'
222 done
'10.1016/j.matdes.2012.10.004'
223 done
'10.1016/j.jclepro.2017.01.040'
224 done
'10.1016/j.jmrt.2021.09.052'
225 done
'10.1016/j.radphyschem

399 done
'10.1016/j.psep.2020.04.044'
400 done
'10.1016/j.jobe.2021.103399'
401 done
'10.1016/j.conbuildmat.2022.127944'
402 done
'10.1016/j.compositesb.2021.108876'
403 done
'10.1016/j.matpr.2020.01.259'
404 done
'10.1016/j.matpr.2020.05.176'
405 done
'10.1016/j.procir.2019.04.337'
406 done
'10.1016/j.conbuildmat.2020.121771'
407 done
'10.1016/j.wasman.2021.03.005'
408 done
'10.1016/j.jclepro.2019.119535'
409 done
'10.1016/j.matpr.2020.04.287'
410 done
'10.1016/j.wasman.2020.10.011'
411 done
'10.1016/j.matpr.2020.12.278'
412 done
'10.1016/j.cscm.2019.e00227'
413 done
'10.1016/j.resconrec.2020.105014'
414 done
'10.1016/j.jclepro.2020.121988'
415 done
'10.1016/j.jmrt.2020.09.038'
416 done
'10.1016/j.cscm.2022.e00934'
417 done
'10.1016/j.apcatb.2022.121667'
418 done
'10.1016/j.apcata.2022.118681'
419 done
'10.1016/j.aiepr.2019.05.002'
420 done
'10.1016/j.conbuildmat.2021.123377'
421 done
'10.1016/j.chroma.2019.05.044'
422 done
'10.1016/j.conbuildmat.2020.121427'
423 done
'10.1016/j.jclep

597 done
'10.1016/j.molliq.2017.10.116'
598 done
'10.1016/j.apcatb.2022.121597'
599 done
'10.1016/j.matpr.2021.10.054'
600 done
'10.1016/j.polymertesting.2015.11.020'
601 done
'10.1016/j.polymertesting.2009.12.010'
602 done
'10.1016/j.biotechadv.2022.107991'
603 done
'10.1016/j.jhazmat.2022.129709'
604 done
'10.1016/j.polymer.2022.125511'
605 done
'10.1016/j.matdes.2014.01.051'
606 done
'10.1016/j.jiec.2022.12.009'
607 done
'10.1016/j.polymdegradstab.2015.01.012'
608 done
'10.1016/j.resconrec.2021.105440'
609 done
'10.1016/j.pce.2022.103264'
610 done
'10.1016/j.jhazmat.2022.130407'
611 done
'10.1016/j.susmat.2022.e00412'
612 done
'10.1016/j.tca.2017.02.013'
613 done
'10.1016/j.eti.2022.102453'
614 done
'10.1016/j.scitotenv.2022.156038'
615 done
'10.1016/j.polymertesting.2021.107273'
616 done
'10.1016/j.matpr.2022.04.353'
617 done
'10.1016/j.jtte.2022.07.002'
618 done
'10.1016/j.matpr.2021.11.459'
619 done
'10.1016/j.wasman.2014.10.017'
620 done
'10.1016/j.matdes.2009.12.028'
621 done
'

796 done
'10.1016/j.jestch.2020.07.006'
797 done
'10.1016/j.resconrec.2022.106279'
798 done
'10.1016/j.conbuildmat.2022.128633'
799 done
'10.1016/j.aiepr.2021.02.001'
800 done
'10.1016/j.jhazmat.2022.130287'
801 done
'10.1016/j.jobe.2022.105203'
802 done
'10.1016/j.apenergy.2022.120564'
803 done
'10.1016/j.wasman.2021.02.006'
804 done
'10.1016/j.catcom.2016.01.036'
805 done
'10.1016/j.msea.2005.05.060'
806 done
'10.1016/j.conbuildmat.2019.05.172'
807 done
'10.1016/j.ces.2022.117875'
808 done
'10.1016/j.cjche.2021.10.028'
809 done
'10.1016/j.tetlet.2005.09.082'
810 done
'10.1016/j.radphyschem.2020.109079'
811 done
'10.1016/j.clema.2022.100113'
812 done
'10.1016/j.matdes.2011.03.037'
813 done
'10.1016/j.susmat.2020.e00194'
814 done
'10.1016/j.apcatb.2022.122302'
815 done
'10.1016/j.est.2022.106329'
816 done
'10.1016/S0141-3910(01)00012-X'
817 done
'10.1016/j.polymdegradstab.2007.07.022'
818 done
'10.1016/j.solener.2021.10.079'
819 done
'10.1016/j.resconrec.2021.105640'
820 done
'10.1016/

...

5057 done
'10.1016/j.conbuildmat.2017.12.048'
5058 done
'10.1016/B978-188420775-4.50023-4'
5059 done
'10.1016/j.seppur.2018.05.004'
5060 done
'10.1016/j.conbuildmat.2014.06.001'
5061 done
'10.1533/9780857098979.58'
5062 done
'10.1016/j.biortech.2013.06.089'
5063 done
'10.1016/j.phpro.2011.11.001'
5064 done
'10.1016/j.wasman.2016.08.039'
5065 done
'10.1016/S0014-3057(01)00272-5'
5066 done
'10.1016/j.ijhydene.2017.02.034'
5067 done
'10.1016/j.wasman.2010.05.003'
5068 done
'10.1016/j.jcis.2017.02.016'
5069 done
'10.1016/j.fuproc.2009.10.009'
5070 done
'10.1016/j.cej.2012.07.058'
5071 done
'10.1016/j.polymertesting.2008.06.001'
5072 done
'10.1016/S0016-2361(01)00067-9'
5073 done
'10.1533/9780857096906.4.500'
5074 done
'10.1016/j.apradiso.2016.01.016'
5075 done
'10.1533/9781845697662.3.303'
5076 done
'10.1016/B978-0-12-820352-1.00235-2'
5077 done
'10.1016/j.anifeedsci.2010.11.016'
5078 done
'10.1016/B978-0-12-814240-0.00023-9'
5079 done
'10.1016/j.jaap.2014.09.027'
5080 done
'10.1016/j.resc

5250 done
'10.1016/j.biortech.2018.02.109'
5251 done
'10.1016/j.conbuildmat.2016.01.056'
5252 done
'10.1016/j.polymdegradstab.2012.04.002'
5253 done
'10.1016/j.biortech.2015.04.016'
5254 done
'10.1016/j.colsurfa.2006.10.043'
5255 done
'10.1016/j.jclepro.2017.08.179'
5256 done
'10.1016/j.triboint.2022.107785'
5257 done
'10.1016/j.jclepro.2015.06.001'
5258 done
'10.1016/j.vibspec.2020.103105'
5259 done
'10.1016/j.conbuildmat.2017.05.122'
5260 done
'10.1016/j.msea.2010.07.007'
5261 done
'10.1016/j.molliq.2016.11.054'
5262 done
'10.1016/j.matdes.2013.01.069'
5263 done
'10.1016/j.compositesb.2014.06.007'
5264 done
'10.1016/j.indcrop.2018.03.070'
5265 done
'10.1016/S0141-3910(03)00160-5'
5266 done
'10.1016/j.jclepro.2015.10.029'
5267 done
'10.1016/j.bej.2017.08.015'
5268 done
'10.1016/B978-0-85709-843-6.00018-4'
5269 done
'10.1016/j.foodchem.2019.125792'
5270 done
'10.1016/j.jconrel.2022.05.020'
5271 done
'10.1016/j.agwat.2011.11.017'
5272 done
'10.1016/j.jaap.2009.07.008'
5273 done
'10.1016

5443 done
'10.1016/j.conbuildmat.2012.09.049'
5444 done
'10.1016/j.mineng.2011.09.019'
5445 done
'10.1016/j.polymdegradstab.2007.04.010'
5446 done
'10.1016/j.micromeso.2006.05.027'
5447 done
'10.1016/j.molcata.2006.10.030'
5448 done
'10.1016/B978-1-4557-3145-9.00006-3'
5449 done
'10.1016/j.polymdegradstab.2004.07.017'
5450 done
'10.1016/j.polymer.2017.07.027'
5451 done
'10.1016/j.polymertesting.2011.10.003'
5452 done
'10.1016/j.jobe.2022.104429'
5453 done
'10.1016/j.resconrec.2018.08.013'
5454 done
'10.1016/j.jclepro.2017.09.232'
5455 done
'10.1016/j.polymertesting.2021.107429'
5456 done
'10.1016/j.envint.2012.11.006'
5457 done
'10.1016/j.psep.2019.02.018'
5458 done
'10.1016/B978-0-08-102676-2.00015-3'
5459 done
'10.1016/j.apcata.2012.04.019'
5460 done
'10.1016/j.seppur.2011.12.028'
5461 done
'10.1016/j.molliq.2018.01.094'
5462 done
'10.1016/j.jmatprotec.2011.01.021'
5463 done
'10.1016/j.polymer.2016.02.004'
5464 done
'10.1016/j.jwpe.2014.08.011'
5465 done
'10.1016/j.compchemeng.2011.0

5636 done
'10.1016/S0167-2991(99)80120-7'
5637 done
'10.1016/j.rsma.2022.102777'
5638 done
'10.1016/j.polymdegradstab.2006.01.012'
5639 done
'10.1016/j.jclepro.2015.07.119'
5640 done
'10.1016/j.wasman.2006.06.007'
5641 done
'10.1016/j.polymer.2012.03.055'
5642 done
'10.1016/j.molliq.2016.08.052'
5643 done
'10.1016/j.scitotenv.2022.160846'
5644 done
'10.1016/S0167-2991(99)80249-3'
5645 done
'10.1016/j.envpol.2010.08.034'
5646 done
'10.1016/j.cplett.2021.138482'
5647 done
'10.1016/j.jaap.2017.03.013'
5648 done
'10.1016/S1389-0344(03)00063-7'
5649 done
'10.1016/j.scitotenv.2018.07.276'
5650 done
'10.1016/j.marpolbul.2012.02.006'
5651 done
'10.1016/j.eurpolymj.2011.05.006'
5652 done
'10.1016/j.polymer.2018.08.066'
5653 done
'10.1016/j.scitotenv.2013.04.056'
5654 done
'10.1016/0269-7483(90)90165-O'
5655 done
'10.1016/j.scitotenv.2017.02.052'
5656 done
'10.1016/S1385-8947(00)00347-8'
5657 done
'10.1016/B978-0-12-803581-8.11278-0'
5658 done
'10.1016/j.polymer.2016.12.054'
5659 done
'10.1016/j

5829 done
'10.1016/j.chemosphere.2007.04.069'
5830 done
'10.1016/j.jhazmat.2014.02.031'
5831 done
'10.1016/j.matchemphys.2014.08.029'
5832 done
'10.1016/j.ibiod.2016.01.025'
5833 done
'10.1016/j.eurpolymj.2004.11.033'
5834 done
'10.1016/j.susmat.2020.e00193'
5835 done
'10.1016/j.surfcoat.2004.03.035'
5836 done
'10.1016/S0969-806X(02)00470-X'
5837 done
'10.1016/j.polymdegradstab.2005.03.016'
5838 done
'10.1016/j.aiepr.2022.08.006'
5839 done
'10.1016/S0076-6879(05)03062-4'
5840 done
'10.1016/j.ctta.2023.100105'
5841 done
'10.1016/j.jconhyd.2010.11.003'
5842 done
'10.1016/j.cemconcomp.2007.02.002'
5843 done
'10.1016/j.apcatb.2011.08.009'
5844 done
'10.1016/j.apenergy.2017.05.141'
5845 done
'10.1016/j.polymer.2006.12.038'
5846 done
'10.1016/j.molcata.2008.01.021'
5847 done
'10.1016/j.jaridenv.2011.05.010'
5848 done
'10.1016/j.jclepro.2018.03.021'
5849 done
'10.1016/j.jaap.2008.10.007'
5850 done
'10.1016/j.radphyschem.2007.03.017'
5851 done
'10.1016/j.petrol.2017.07.007'
5852 done
'10.1016/

In [20]:
df_available_abstract = pd.DataFrame()
df_available_abstract['DOI'] = list_doi
df_available_abstract['Title'] = list_title
df_available_abstract['Abstract'] = list_abstract
df_available_abstract['Date'] = list_date
df_available_abstract['Journal'] = list_journal

df_available_abstract.head(10)

DOI  \
0       10.1016/j.susmat.2022.e00562   
1         10.1016/j.jobe.2022.105541   
2      10.1016/j.polymer.2022.125596   
3         10.1016/j.fpsl.2022.101020   
4         10.1016/j.egyr.2022.07.101   
5    10.1016/j.resconrec.2022.106444   
6         10.1016/j.jics.2022.100843   
7        10.1016/j.clwas.2022.100044   
8         10.1016/j.jmrt.2022.10.126   
9  10.1016/j.conbuildmat.2021.126118   

                                               Title  \
0  Environmental assessment of the recycling proc...   
1  Sustainable utilization of hybrid recycled pow...   
2  Simulations mapping the influence of oxygen, e...   
3  Migration of volatile substances from recycled...   
4  Effect of compatibilization agents and rice hu...   
5  Bioleaching/enzyme-based recycling of aluminiu...   
6  Recycling of polyethylene terephthalate (PET O...   
7  Powder production from recycled low-density po...   
8  High strength insulating polymeric composite b...   
9  Structural characterization and application of...   

                                            Abstract                    Date  \
0  The plastic packaging, and in particular the m...              April 2023   
1  The effects of hybrid recycled powder (HRP) fr...          1 January 2023   
2  We present a computational study of the develo...         16 January 2023   
3  Plastic recycling is the main solution to redu...              March 2023   
4  The new trend of the circular economy encourag...           November 2022   
5  Multilayer materials are frequently used in fo...            October 2022   
6  Waste management is become one of the world's ...            January 2023   
7  The circular economy approach has been establi...           December 2022   
8  In the current work, post-consumer polyethylen...  November–December 2022   
9  The use of recycled polyethylene as a bitumen ...         17 January 2022   

                                        Journal  
0        Sustainable Materials and Technologies  
1               Journal of Building Engineering  
2                                       Polymer  
3                 Food Packaging and Shelf Life  
4                                Energy Reports  
5         Resources, Conservation and Recycling  
6        Journal of the Indian Chemical Society  
7                         Cleaner Waste Systems  
8  Journal of Materials Research and Technology  
9           Construction and Building Materials

In [21]:
df_available_abstract.shape

(6000, 5)

In [22]:
df_available_abstract.to_csv('pet_----------------------1.csv')